In [1]:
import requests as rq
import pandas as pd
import re
import json
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
types = ['IN', 'MS', 'PU', 'SP']
years = []
for year in range(1997, 2022):
    previous = str(year - 1)[2:]
    now = str(year)[2:]
    years.append(previous + now)

In [5]:
# добавить колонку с дистанцией
results = []
for evtype in tqdm(['BTSWRLCH', 'BTSWRLCP']): 
    for discipline in tqdm(types):
        for year in years:
            coms = rq.get('https://biathlonresults.com/modules/sportapi/api/SearchResults?SeasonId=' + year + 
                          '&EventTypeId=' + evtype +
                          '&DisciplineId=' + discipline).json()
            for comp in coms:
                race_id = comp['RaceId']
                description = comp['Description']
                ev_type = comp['ShortEventType']
                place = comp['Place']
                dt_not_form = datetime.strptime(comp['StartTime'], "%Y-%m-%dT%H:%M:%S")
                dt_form = f"{dt_not_form:%d-%m-%Y}"
                
                data = rq.get('https://biathlonresults.com/modules/sportapi/api/Results?RT=385698&RaceId=' + race_id).json()
                
                for athlete in range(0, len(data['Results'])):
                    result = data['Results'][athlete]
                    
                    result['Distance'] = data['Competition']['km']
                    
                    result['Date'] = dt_form
                    result['Event Type'] = ev_type
                    result['Place'] = place
                    result['Type'] = discipline
                    
                    results.append(result)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [05:57<00:00, 178.84s/it]


In [6]:
res = pd.DataFrame(results)[['IBUId', 'FamilyName', 'GivenName', 'Shootings', 'ShootingTotal', 'Result', 
                             'StartOrder','ResultOrder', 'Date', 'Event Type', 'Place', 'Type', 'WC', 'Distance']]
res

,IBUId,FamilyName,GivenName,Shootings,ShootingTotal,Result,StartOrder,ResultOrder,Date,Event Type,Place,Type,WC,Distance
0,BTGER12208197001,GROSS,Ricco,0+0+0+0,0,52:04.6,1,1,07-02-1997,WCH,Brezno-Osrblie (SVK),IN,None,20
1,BTBLR11512196701,RYZHENKOV,Oleg,1+0+0+1,2,+35.5,2,2,07-02-1997,WCH,Brezno-Osrblie (SVK),IN,None,20
2,BTAUT11911196701,GREDLER,Ludwig,0+0+2+0,2,+1:08.4,3,3,07-02-1997,WCH,Brezno-Osrblie (SVK),IN,None,20
3,BTRUS10104197201,ROZHKOV,Sergei,0+1+0+0,1,+1:11.3,4,4,07-02-1997,WCH,Brezno-Osrblie (SVK),IN,None,20
4,BTGER11604197101,FISCHER,Sven,1+0+0+1,2,+1:44.9,5,5,07-02-1997,WCH,Brezno-Osrblie (SVK),IN,None,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89466,BTBUL10607200101,TODEV,Blagoy,2+2,4,+5:35.9,98,97,29-11-2020,WC,Kontiolahti (FIN),SP,,10
89467,BTSWE10404200001,STEFANSSON,Malte,4+3,7,+5:37.9,88,98,29-11-2020,WC,Kontiolahti (FIN),SP,,10
89468,BTKAZ10811199401,SIRIK,Sergey,2+3,5,+5:54.1,91,99,29-11-2020,WC,Kontiolahti (FIN),SP,,10
89469,BTLTU11210200001,FOMIN,Maksim,2+2,4,+5:57.3,83,100,29-11-2020,WC,Kontiolahti (FIN),SP,,10


In [6]:
people = []
for id_person in tqdm(set(res.IBUId)):
    person = rq.get('https://biathlonresults.com//modules/sportapi/api/CISBios?IBUId='+ id_person).json()
    
    gender = person['GenderId']
    nationality = person['NAT']
    f_name = person['FamilyName']
    g_name = person['GivenName']
    
    birth = person['Birthdate']
    if birth != None:
        birth = datetime.strptime(birth, '%Y-%m-%dT%H:%M:%S')
    
    personal = person['Personal']
    
    atr = [list(i.values())[1] for i in personal]
    val = [list(i.values())[2] for i in personal]
    
    person = {'FamilyName': f_name, 'GivenName': g_name, 'DateOfBirth': birth, 'Gender': gender, 'Nationality': nationality}
    person = person | dict(list(zip(atr, val)))
    
    people.append(person)

100%|██████████████████████████████████████████████████████████████████████████████| 1803/1803 [08:59<00:00,  3.34it/s]


In [7]:
people_r = pd.DataFrame(people)
people_r = people_r.drop(['Age'], axis = 1)

In [7]:
res.to_csv('results.csv')

In [8]:
people_r.to_csv('people.csv')